In [1]:
import numpy as np
import pandas as pd
import zipfile as zp
pd.set_option('display.max_columns', 1000)
from myfeatures import engineer
import warnings
warnings.filterwarnings("ignore")

In [2]:
with zp.ZipFile('train_zsTMYVA.zip','r') as tr, zp.ZipFile('test_9tJUnaB.zip', 'r') as te:
    train = pd.read_csv(tr.open(tr.namelist()[0]))
    test = pd.read_csv(te.open(te.namelist()[0]))

In [3]:
## need to normalise variables

def mungeNumerics(train, test, numeric_feats):
    
    ntrain = train.shape[0]
    test['Responders'] = -1
    
    train_test = pd.concat((train, test)).reset_index(drop=True)
    
    ## compute skew do log transformation
    for x in numeric_feats:
        calskew = train_test[x].skew()
        if calskew > 0.25 or calskew < 0.25:
            minimum = min(train_test[x])
            if minimum <= 0:
                shift = abs(minimum) +1
                train_test[x] = train_test[x].map(lambda k: np.log10(k + shift))
            else:
                train_test[x] = train_test[x].map(lambda k: np.log10(k))
        
    return train_test

In [4]:
numericfeat = train.select_dtypes(include=[np.float64, np.int64]).columns
numericfeat = [x for x in numericfeat if x not in ['UCIC_ID','Responders']]

In [6]:
train.head()

,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_Pr

In [5]:
numericfeat

['NO_OF_Accs',
 'vintage',
 'dependents',
 'zip',
 'C_prev1',
 'D_prev1',
 'ATM_C_prev1',
 'ATM_D_prev1',
 'BRANCH_C_prev1',
 'BRANCH_D_prev1',
 'IB_C_prev1',
 'IB_D_prev1',
 'MB_C_prev1',
 'MB_D_prev1',
 'POS_C_prev1',
 'POS_D_prev1',
 'count_C_prev1',
 'count_D_prev1',
 'COUNT_ATM_C_prev1',
 'COUNT_ATM_D_prev1',
 'COUNT_BRANCH_C_prev1',
 'COUNT_BRANCH_D_prev1',
 'COUNT_IB_C_prev1',
 'COUNT_IB_D_prev1',
 'COUNT_MB_C_prev1',
 'COUNT_MB_D_prev1',
 'COUNT_POS_C_prev1',
 'COUNT_POS_D_prev1',
 'custinit_CR_amt_prev1',
 'custinit_DR_amt_prev1',
 'custinit_CR_cnt_prev1',
 'custinit_DR_cnt_prev1',
 'ATM_amt_prev1',
 'ATM_CW_Amt_prev1',
 'ATM_CW_Cnt_prev1',
 'BRN_CW_Amt_prev1',
 'BRN_CW_Cnt_prev1',
 'BRN_CASH_Dep_Amt_prev1',
 'BRN_CASH_Dep_Cnt_prev1',
 'CNR_prev1',
 'BAL_prev1',
 'EOP_prev1',
 'CR_AMB_Prev1',
 'C_prev2',
 'D_prev2',
 'ATM_C_prev2',
 'ATM_D_prev2',
 'BRANCH_C_prev2',
 'BRANCH_D_prev2',
 'IB_C_prev2',
 'IB_D_prev2',
 'MB_C_prev2',
 'MB_D_prev2',
 'POS_C_prev2',
 'POS_D_prev2',
 

In [38]:
alldata = mungeNumerics(train, test, numericfeat)

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:19: RuntimeWarning: divide by zero encountered in log10


In [40]:
test = alldata[alldata['Responders'] == -1]
train = alldata[alldata['Responders'] != -1]

In [42]:
test.drop('Responders', axis=1, inplace=True)

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
print(train.shape)
print(test.shape)

(300000, 377)
(200000, 376)


In [44]:
train, test = engineer(train, test)

/home/manish/Desktop/Data2017/september/churn_prediction/myfeatures.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train['total_credit_amt'] = train['C_prev1'] + train['C_prev2'] + train['C_prev3'] + train['C_prev3'] + train['C_prev5'] + train['C_prev6']
/home/manish/Desktop/Data2017/september/churn_prediction/myfeatures.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['total_credit_amt'] = test['C_prev1'] + test['C_prev2'] + test['C_prev3'] + test['C_prev3'] + test['C_prev5'] + test['C_prev6']
/home/manish/Desktop/Data2017/s

In [46]:
for x in train.select_dtypes(include=['object']).columns:
    if train[x].nunique() != test[x].nunique():
        set_train = set(train[x].unique())
        set_test = set(test[x].unique())
        
        remove_train = set_train - set_test
        remove_test = set_test - set_train
        
        remove = remove_train.union(remove_test)
        
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        
        train[x] = train[x].map(filter_cat)
        test[x] = test[x].map(filter_cat)

In [47]:
colMiss = train.isnull().sum(axis = 0) / train.shape[0]

toremove = []
for k,v in colMiss.items():
    if v == 1:
        toremove.append(k)

In [48]:
train.drop(toremove, axis=1, inplace=True)
test.drop(toremove, axis=1, inplace=True)

In [49]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [50]:
for c in train.select_dtypes(include=['object','category']).columns:
    if c not in ['UCIC_ID','Responders']:
        # print(c)
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [51]:
feature_names = [col for col in train.columns if col not in ['UCIC_ID','Responders']]
target = train['Responders']

In [52]:
## default
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1, # 0.1
          "max_depth": 6, # 7
          "subsample": 1,
          "colsample_bytree": 1,
          "seed": 2016, 
         "eval_metric":"auc"}

In [54]:
from sklearn.model_selection import StratifiedKFold
n_folds = 5
skf = StratifiedKFold(n_splits = n_folds, random_state=420)

In [55]:
def xgbStackerModel(train, test, target, features):
    
    oof_pred = pd.DataFrame({'UCIC_ID':train['UCIC_ID'], 'Responders':0})
    all_predictions = pd.DataFrame()
    
    increase = True
    best_rounds = []
    
    for i, (train_index, valid_index) in enumerate(skf.split(train, target)):
        print('[{}/{} Folds]'.format(i+1, n_folds))
        
        X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        
        
        if increase:
            pos = pd.Series(target == 1)
            
            X_train = pd.concat([X_train, X_train[pos]], axis=0)
            y_train = pd.concat([y_train, y_train[pos]], axis=0)
            
            idx = np.arange(len(X_train))
            np.random.shuffle(idx)
            
            X_train = X_train.iloc[idx]
            y_train = y_train.iloc[idx]
            
        dtrain = xgb.DMatrix(X_train[features], y_train, missing=np.nan)
        dvalid = xgb.DMatrix(X_valid[features], y_valid, missing=np.nan)
        dtest = xgb.DMatrix(test[features])
            
        watchlist = [(dtrain, 'train'),(dvalid, 'valid')]
        clf1 = xgb.train(params, dtrain, num_boost_round=1000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)    
        
        best_rounds.append(clf1.best_iteration)
        
        preds1 = clf1.predict(dvalid)
        oof_pred.loc[valid_index, 'Responders'] = preds1
        
        
    ## for test, predict on whole data
    Ndtrain = xgb.DMatrix(data=train[features], label = target, missing=np.nan)
    
    n_round = np.round(np.mean(best_rounds))
    clf2 = xgb.train(params, Ndtrain, n_round)
    preds2 = clf2.predict(dtest)
    
    all_predictions['pred' + str(i)] = preds2
    
    return oof_pred, all_predictions

In [56]:
oof_train, test_pred = xgbStackerModel(train, test, target, feature_names)

[1/5 Folds]
[0]	train-auc:0.83232	valid-auc:0.83161
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.859445	valid-auc:0.854792
[40]	train-auc:0.872716	valid-auc:0.864175
[60]	train-auc:0.880375	valid-auc:0.867126
[80]	train-auc:0.885641	valid-auc:0.868439
[100]	train-auc:0.889926	valid-auc:0.86919
[120]	train-auc:0.893599	valid-auc:0.86952
[140]	train-auc:0.896879	valid-auc:0.869542
[160]	train-auc:0.897878	valid-auc:0.8695
Stopping. Best iteration:
[127]	train-auc:0.894947	valid-auc:0.869573

[2/5 Folds]
[0]	train-auc:0.832065	valid-auc:0.831658
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.859633	valid-auc:0.856426
[40]	train-auc:0.872086	valid-auc:0.863742
[60]	train-auc:0.879625	valid-auc:0.866527
[80]	train-auc:0.885845	valid-auc:0.868372
[100]	train-auc:0.890

KeyboardInterrupt: 

In [ ]:
test_pred['UCIC_ID'] = test['UCIC_ID']
test_pred.rename(columns = {'pred4':'Responders'}, inplace=True)

In [ ]:
oof_train.to_csv('l1/xgb_train.csv', index=False)
test_pred.to_csv('l1/xgb_test.csv', index=False)